In [5]:
import torch.nn as nn
import torch
from torchvision.models import convnext_small

class ClothesClassifier(nn.Module):
    def __init__(self, num_classes, pretrained=False):
        super(ClothesClassifier, self).__init__()
        freeze_ratio = 0.8
        self.model = convnext_small(weights="IMAGENET1K_V1" if pretrained else None)

        num_stages = len(self.model.features)
        num_stages_to_freeze = int(freeze_ratio * num_stages)

        for stage_idx in range(num_stages_to_freeze):
            for param in self.model.features[stage_idx].parameters():
                param.requires_grad = False

        in_features = self.model.classifier[2].in_features
        self.model.classifier[2] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [6]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

def load_model(model_path, num_classes, device="cuda"):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file '{model_path}' not found.")
    model = ClothesClassifier(num_classes=num_classes)
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    model.to(device)
    model.eval()
    return model


In [7]:
def preprocess_image(image_path):
    """
    Preprocess an image for model input.
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image


In [8]:
def predict(image_path, model, category_mapping, device="cuda"):
   
    image = preprocess_image(image_path).to(device)
    
    with torch.no_grad():
        outputs = model(image)
    
    predicted_class = torch.argmax(outputs, dim=1).item()
    
    # Reverse mapping from index to class name
    idx_to_category = {v: k for k, v in category_mapping.items()}
    predicted_category = idx_to_category.get(predicted_class, "Unknown")
    
    print(f"Predicted Category: {predicted_category}")

In [20]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print (f"Using device: {device}")
# Example usage
model_path = "C:/Users/DELL/Downloads/model_weights_last.pth"  # Update with your actual path
num_classes = 7  # Use the actual number of classes from training
image_path = "C:/Users/DELL/OneDrive/Pictures/Wardrobe/png-transparent-handbag-designer-woman-women-s-handbags-white-luggage-bags-holidays-thumbnail.png"

# Mapping category indices to names (from dataset processing)
category_mapping = {'Bags': 0, 'Bottomwear': 1, 'Headwear': 2, 'Jewellery': 3, 'Shoes': 4, 'Topwear': 5, 'Watches': 6}

# Load the model with the correct device
model = load_model(model_path, num_classes, device=device)

# Print the model architecture
# Perform prediction
predict(image_path, model, category_mapping, device=device)

Predicted Category: Bags
